In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.afw.cameraGeom as cameraGeom
import lsst.afw.cameraGeom.utils as cgUtils

In [ ]:
afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
from lsst.obs.lsst import LsstCamMapper as camMapper
camera = camMapper._makeCamera()

## Routines to read per-amplifier images and convert coordinates to/from CCD pixel coordinates

In [ ]:
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms, channelToAmp, getAmpImage

### Put in a raft,detector name by hand

In [ ]:
raftName = 'R22'
detectorName = 'S11'
detectorName = "%s_%s" % (raftName, detectorName)

lct = LsstCameraTransforms(camera)

#### Get a DM detector and ask it about itself

In [ ]:
det = lct.getDetector(detectorName)
print(det.getName(), det.getType(), det.getPhysicalType(), det.getSerial())

#### Convert some CCD coordinates to amp coordinates

Validate this by looking at some images with recognisable pixels

In [ ]:
# Choose some pixels to check by eye
cx, cy = 3332, 2500   # C16

#
# Convert from the CCD coordinates to per-amp coordinates
#
channel, ampX, ampY = lct.ccdPixelToAmpPixel(cx, cy, detectorName)

butler = None
if butler is not None:
    #
    # Read the raw amp data, assuming that the butler isn't adding any information
    #
    rawa = getAmpImage(butler, dataId, channel)

    #
    # Display the amp image, showing the selected point
    #
    disp = afwDisplay.Display(4, reopenPlot=True)

    stats = afwMath.makeStatistics(rawa.image, afwMath.MEDIAN | afwMath.STDEVCLIP)
    med = stats.getValue(afwMath.MEDIAN)
    std = stats.getValue(afwMath.STDEVCLIP)
    disp.scale('linear', med - 1*std, med + 2*std)

    disp.mtv(rawa, title=channelToAmp(lct.getDetector(detectorName), channel).getName())
    disp.dot('+', ampX, ampY, ctype=afwDisplay.RED)
    disp.zoom(16, ampX, ampY)

In [ ]:
print(channel, ampX, ampY)

### Plot the whole focal plane

In [ ]:
plt.close('all')
cgUtils.plotFocalPlane(camera)
plt.title(camera.getName());

#### Convert a CCD position to focal plane coordinates and plot it

In [ ]:

for ccdXY, color in [
    ((-0.5, -0.5), "red"),   # bottom left corner of CCD
    (camera[detectorName].getBBox().getEnd() - afwGeom.ExtentD(0.501, 0.501), "green"),  # top right of CCD
    ]:
    fpXMm, fpYMm = lct.ccdPixelToFocalMm(*ccdXY, detectorName)

    plt.plot(fpXMm, fpYMm, '+', color=color)
    print("%s (%8.3f, %8.3f)pix -> (%8.3f, %8.3f)mm" % (detectorName, ccdXY[0], ccdXY[1], fpXMm, fpYMm))

In [ ]:
ccdXY = [0.,0.]    
fpXMm, fpYMm = lct.ccdPixelToFocalMm(*ccdXY, detectorName)
plt.plot(fpXMm, fpYMm, '+', color=color)
print("%s (%8.3f, %8.3f)pix -> (%8.3f, %8.3f)mm" % (detectorName, ccdXY[0], ccdXY[1], fpXMm, fpYMm))

In [ ]:
camera[detectorName].getBBox().getEnd()

#### Demonstrate the reverse mapping;  focal plane back to CCD

In [ ]:
dname, ccdX, ccdY = lct.focalMmToCcdPixel(fpXMm, fpYMm)
print("(%.3f, %.3f)mm -> %s (%.3f, %.3f)pix" % (fpXMm, fpYMm, dname, ccdX, ccdY))

In [ ]:
dname = 'R22_S11'
cx, cy=0,0
lct.ccdPixelToAmpPixel(cx,cy,dname)
#cx, cy= 1018,0
#lct.ccdPixelToAmpPixel(cx,cy,dname)

#### And focal plane to amplifier

In [ ]:
fpXMm, fpYMm = 2.5, 10
dname, channel, ampX, ampY = lct.focalMmToAmpPixel(fpXMm, fpYMm)
print("(%.3f, %.3f)mm -> %s channel %d (%.3f, %.3f)pix" % (fpXMm, fpYMm, dname, channel, ampX, ampY))

In [ ]:
a = 2
b = 0
c = 1
d = 1

x_cam = 127.00*(a - 2) + 42.25*(c - 1)
y_cam = 127.00*(b - 2) + 42.25*(d - 1)
print(x_cam, y_cam)
fpXMm, fpYMm = y_cam, x_cam
dname, channel, ampX, ampY = lct.focalMmToAmpPixel(fpXMm, fpYMm)
print("(%.3f, %.3f)mm -> %s channel %d (%.3f, %.3f)pix" % (fpXMm, fpYMm, dname, channel, ampX, ampY))

In [ ]:
if butler is not None:
    #
    # Read the raw amp data, assuming that the butler isn't adding any information
    #
    rawa = getAmpImage(butler, dataId, channel)

    #
    # Display the amp image, showing the selected point
    #
    disp = afwDisplay.Display(5, reopenPlot=True)

    disp.scale('asinh', 'zscale', Q=2)

    disp.mtv(rawa, title=channelToAmp(lct.getDetector(detectorName), channel).getName())
    disp.dot('+', ampX, ampY, ctype=afwDisplay.RED)
    disp.zoom(32, ampX, ampY)